<a href="https://colab.research.google.com/github/sunjin7725/SJ-SOLAR-10.7b-DPO/blob/main/DPO_apater.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Requirements

In [ ]:
!pip install transformers accelerate datasets trl peft bitsandbytes

# Load Model & Datasets

In [ ]:
dataset_name = 'SJ-Donald/orca-dpo-pairs-ko'
model_name = 'SJ-Donald/SOLAR-10.7B-slerp'
device_map = 'auto'

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset(dataset_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['system', 'question', 'chosen', 'rejected'],
        num_rows: 36009
    })
})

In [ ]:
import torch

from trl import DPOTrainer
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,
    TrainerCallback, TrainingArguments, TrainerState, TrainerControl
    )

from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_int8_training,
    set_peft_model_state_dict
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name, # location of saved SFT model
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
    device_map = device_map
)

Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
print(type(model))
print(model)
print("length of tokenizer:",len(tokenizer))

<class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(48000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaR

In [ ]:
bos = tokenizer.bos_token_id
eos = tokenizer.eos_token_id
pad = tokenizer.pad_token_id
print("pre-trained model's BOS EOS and PAD token id:",bos,eos,pad," => It should be 1 2 None")

pre-trained model's BOS EOS and PAD token id: 1 2 2  => It should be 1 2 None


In [ ]:
tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
tokenizer.padding_side = "right"

In [ ]:
def return_prompt_and_responses(samples):

    return {
        "prompt": "### User:\n" + samples["question"] + "\n\n### Assistant:\n",
        "chosen": samples["chosen"],
        "rejected": samples["rejected"],
    }

cutoff_len = 4096
train_dataset = dataset.map(return_prompt_and_responses)
train_dataset = train_dataset.filter(
    lambda x: len(x["question"]) + len(x["chosen"]) <= cutoff_len
    and len(x["question"]) + len(x["rejected"]) <= cutoff_len
)
train_dataset = train_dataset["train"].shuffle()

print(train_dataset['question'][0])
print(train_dataset['chosen'][0])
print(train_dataset['rejected'][0])

다음 질문에 답하세요: 다음 기사에 대한 객관식 문제를 작성하십시오: 기사: 욕실이 종종 박테리아로 가득 차 있다고 생각하여 일주일에 한 번 완전히 청소합니다. 그러나 냉장고는 얼마나 자주 청소하고 소독하나요? 최근 연구에 따르면 냉장고가 화장실에서 두 번째로 가장 더럽은 집안 공간으로, 냉장고의 온도를 너무 높게 설정해 박테리아가 증식할 수 있어 식중독을 쉽게 입수할 수 있습니다. 따라서 냉장고와 냉장고 내 음식이 건강에 위협이 되지 않도록 하려면 어떻게 해야 할까요? 우선 매주 냉장고를 청소하세요. 닦고 반복하세요. 냉장고의 온도가 섭씨 0~5도 사이인지 확인하세요. 냉동고는 섭씨 -18도 미만이어야 합니다. 냉장고 및 냉동고 온도계를 사용하여 이것이 사실인지 확인할 수 있습니다. 일부 현대형 냉장고에는 고기와 야채 등 다양한 종류의 음식을 보관할 수 있는 영역이 있습니다. 야채 영역의 온도가 더 높을 수 있으므로 고기를 야채 영역에 보관하지 마세요. 냉장고 내 온도가 급격히 상승하여 박테리아의 번식지가 될 수 있으므로 문을 너무 오래 열어두지 마세요. 완료하지 못한 음식은 덮어두고 며칠 안에 먹어버리세요. 병과 항아리는 닫혀 있어야 합니다. 조리한 고기와 원재료는 오염을 방지하기 위해 잘 덮어져 있어야 합니다. 다른 음식에 육즙이 스며들지 않도록 원재료 고기를 냉장고 바닥에 보관하세요. 마지막으로, 상식적인 사고를 발휘하여 오래된 음식은 버리세요.Answer:
질문: 이 기사에 따르면, 다음 중 냉장고의 깨끗하고 안전한 환경을 유지하는 데 도움이 될 수 있는 행동은?A) 매주 냉장고를 청소하고 올바른 온도를 설정합니다.B) 생고기를 채소 구역에 보관C) 냉장고 문을 오래 열어두는 것D) 원시 고기와 조리된 고기를 같은 저장 공간에 혼합하는 것정답입니다: A) 매주 냉장고를 청소하고 올바른 온도를 설정합니다.
다음 중 냉장고를 정기적으로 청소해야 하는 이유에 대한 설명 중 옳은 것은 무엇인가요?

A) 냉장고는 식중독의 주요 원인이기 때문입니다.
B) 냉장

# DPO adapter train

In [ ]:
# training hyperparams
batch_size = 32
micro_batch_size = 1
num_epochs = 1
learning_rate = 5e-7
cutoff_len = 4096
val_set_size = 0
lr_scheduler = "linear"
warmup_ratio = 0.1
gradient_accumulation_steps = batch_size//micro_batch_size
output_dir="./output"

# lora hyperparams
lora_r = 16
lora_alpha = 16
lora_dropout = 0.05
# from peft docs: ["q_proj", "k_proj", "v_proj", "o_proj", "fc_in", "fc_out", "wte", "gate_proj", "down_proj", "up_proj"]
lora_target_modules = ['embed_tokens', 'q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'lm_head']

# llm hyperparams
train_on_inputs = False  # if False, masks out inputs in loss
add_eos_token = False
group_by_length = False  # faster, but produces an odd training loss curve

In [ ]:
training_args = TrainingArguments(
    num_train_epochs= num_epochs,
    per_device_train_batch_size=micro_batch_size,
    #per_device_eval_batch_size=script_args.per_device_eval_batch_size,
    #max_steps=1000,
    logging_steps=1,
    save_steps=10,
    save_total_limit=2,
    gradient_accumulation_steps=gradient_accumulation_steps,
    #gradient_checkpointing=script_args.gradient_checkpointing,
    learning_rate=learning_rate,
    #evaluation_strategy="steps",
    #eval_steps=script_args.eval_steps,
    output_dir=output_dir,
    #report_to=script_args.report_to,
    lr_scheduler_type=lr_scheduler,
    warmup_ratio=warmup_ratio,
    optim='paged_adamw_32bit', # rmsprop
    bf16=True,
    remove_unused_columns=False,
)

peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
import os
import sys

# DPO trainer
dpo_trainer = DPOTrainer(
    model,
    ref_model = None, #model_ref,
    args=training_args,
    beta=0.1, # fix
    train_dataset=train_dataset,
    #eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
)

if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)

# train
dpo_trainer.train()
dpo_trainer.save_model(output_dir)

# save
output_dir = os.path.join(output_dir, "final_checkpoint")
dpo_trainer.model.save_pretrained(output_dir)

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:282: UserWarning: `max_length` is not set in the DPOTrainer's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:289: UserWarning: `max_prompt_length` is not set in the DPOTrainer's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/36008 [00:00<?, ? examples/s]

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.693100
2,0.693100
3,0.697400
4,0.683800
5,0.695300
6,0.695500
7,0.687400
8,0.690600
9,0.698400
10,0.692300


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers fou

Step,Training Loss
1,0.693100
2,0.693100
3,0.697400
4,0.683800
5,0.695300
6,0.695500
7,0.687400
8,0.690600
9,0.698400
10,0.692300


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers fou